In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

from tensorboardX import SummaryWriter
from tqdm import tqdm as tqdm

CUDA = torch.cuda.is_available()

import numpy as np

from sklearn.metrics import accuracy_score

import torchtext
from collections import Counter

In [2]:
text_field = torchtext.data.Field(
    lower=True, include_lengths=False, fix_length=2048, tensor_type=torch.FloatTensor, batch_first=True,
    tokenize=lambda x: x, use_vocab=False, sequential=False
)
label_field = torchtext.data.Field(sequential=False, use_vocab=False)

# train, test = torchtext.datasets.IMDB.splits(text_field, label_field)
# c = Counter(''.join([' '.join(t.text_field) for t in train]))
# ALPHABET = [char[0] for char in c.most_common(62)]  # all other chars used less ~ 100 times in a test
# ALPHABET.append('UNK')
# ALPHABET.append('PAD')
ALPHABET = [' ', 'e', 't', 'a', 'i', 'o', 's', 'n', 'r', 'h', 'l', 'd', 'c', 'm', 'u', 'f', 'g', 'y', 'b', 'w', 'p',\
            '.', 'v', ',', 'k', "'", '/', '>', '<', '-', '"', 'j', 'x', ')', '(', '!', 'z', 'q', '0', '1', '?', ':',\
            '9', '2', '*', ';', '3', '5', '8', '4', '7', '&', '6', 'é', '\x96', '`', '$', '\x85', '_', '%', '=', '#',\
            'UNK', 'PAD']

ALPHABET_LEN = len(ALPHABET)

char2int = {s: i for s, i in zip(ALPHABET, range(ALPHABET_LEN))}

In [3]:
MAXLEN = 1024

BATCH_SIZE = 32
TEST_SIZE = 100

def one_hot(char):
    zeros = np.zeros(ALPHABET_LEN)
    if char in char2int:
        zeros[char2int[char]] = 1.
    else:
        zeros[char2int['UNK']] = 1.

def preprocess_text_nobatch(text, maxlen=MAXLEN):
    one_hotted_text = np.zeros((maxlen, ALPHABET_LEN))
    for i, char in enumerate(text):
        if i >= MAXLEN:
            break
        one_hotted_text[i, char2int.get(char, char2int['UNK'])] = 1.
    if i < MAXLEN:
        for j in range(i+1, MAXLEN):
            one_hotted_text[j, char2int['PAD']] = 1.

    return torch.FloatTensor(one_hotted_text)

def onehot2text(one_hotted_text, batch_size=None):
    if batch_size is None:
        text = ''
        _, idx = torch.max(one_hotted_text, 1)
        for i in idx:
            symb = ALPHABET[i]
            if symb == 'PAD':
                break
            else:
                text += symb
        return text
    else:
        texts = []
        for text in one_hotted_text:
            texts.append(onehot2text(one_hotted_text, batch_size=None))
        return texts

class CharIMDB(torchtext.datasets.imdb.IMDB):
    
    def __getitem__(self, idx):
        item = super(CharIMDB, self).__getitem__(idx)
        text = item.text
        label = int(item.label == 'pos')
        return preprocess_text_nobatch(text), label

train, test = CharIMDB.splits(text_field, label_field)

dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Model

In [4]:
class CharCNN(nn.Module):
    
    def __init__(self, dropout=0.5):  #, hidden_dim=256, kernel_size=16):
        super(CharCNN, self).__init__()
        
        self.dropout = dropout
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(ALPHABET_LEN, 256, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )               
        self.conv3 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1),
            nn.ReLU()
        )
        self.conv4 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1),
            nn.ReLU()    
        )
        self.conv5 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1),
            nn.ReLU()
        )
        self.conv6 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )   

        self.fc1 = nn.Sequential(
            nn.Linear(8704, 1024),  # MAXLEN = 1024
            nn.ReLU(),
            nn.Dropout(p=dropout)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=dropout)
        )

        self.fc3 = nn.Linear(1024, 2)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)

        # collapse
        x = x.view(x.size(0), -1)
        # linear layer
        x = self.fc1(x)
        # linear layer
        x = self.fc2(x)
        # linear layer
        x = self.fc3(x)
        return x
    
#     def describe(self):
#         return '_char_cnn_%s_%s' % (self.hidden_dim, self.kernel_size)

In [5]:
model = CharCNN()
if CUDA:
    model.cuda()
model.train()

CharCNN(
  (conv1): Sequential(
    (0): Conv1d(64, 256, kernel_size=(7,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(256, 256, kernel_size=(7,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
  )
  (conv4): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
  )
  (conv5): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
  )
  (conv6): Sequential(
    (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=8704, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
  )
  (fc2): Se

# Training

In [6]:
writer = SummaryWriter(comment='_charCNN_BIG')

In [12]:
optimizer = optim.Adam(params=model.parameters(), lr=10**-5)
optimizer.zero_grad()

In [8]:
global_step = 0

In [13]:
%%time

N_EPOCHS = 10

loss_f = F.cross_entropy

for epoch in range(N_EPOCHS):
    for batch_idx, (text, label) in enumerate(dataloader):

        if CUDA:
            text = Variable(text.cuda())
            label = Variable(torch.LongTensor(label).cuda())
        else:
            text = Variable(text)
            label = Variable(torch.LongTensor(label))

        text = text.permute(0, 2, 1)  # (1, 0, 2) for RNN
        prediction = model(text)

        loss = loss_f(prediction, label)

        writer.add_scalar('loss', loss.data[0], global_step=global_step)

        loss.backward()        
        torch.nn.utils.clip_grad_norm(model.parameters(), 1e-1)
        optimizer.step()
        
        if CUDA:
            torch.cuda.synchronize()
        global_step += 1

    # evaluation
    print('Loss after epoch %s:' % epoch)
    print(loss.data[0])
        
    _, idx = torch.max(prediction, 1)
    acc = accuracy_score(label.data.tolist(), idx.data.tolist())
    writer.add_scalar('accuracy_train', acc, global_step=global_step)
    print('In-batch accuracy:', acc)
    
    model.eval()

    predictions = []
    lables = []

    for i in np.random.choice(range(25000), TEST_SIZE, replace=False):
        _test = test[i]
    #     test_texts.append(_test[0])
    #     test_labels.append(_test[1])
        _text = _test[0].unsqueeze(0).permute(0, 2, 1)
        _text = Variable(_text.cuda()) if CUDA else Variable(_text)
        lables.append(_test[1])
        pred = model(_text)
        _, idx = torch.max(pred, 1)
        predictions.append(idx.data[0])

    acc = accuracy_score(lables, predictions)
    print('Test accuracy:', acc)
    writer.add_scalar('accuracy_test', acc, global_step=global_step)
    print()
    model.train()


Loss after epoch 0:
7.510185241699219e-06
In-batch accuracy: 1.0
Test accuracy: 0.86

Loss after epoch 1:
0.0
In-batch accuracy: 1.0
Test accuracy: 0.83

Loss after epoch 2:
0.0
In-batch accuracy: 1.0
Test accuracy: 0.84

Loss after epoch 3:
5.960464477539063e-08
In-batch accuracy: 1.0
Test accuracy: 0.85

Loss after epoch 4:
3.2782554626464844e-06
In-batch accuracy: 1.0
Test accuracy: 0.76

Loss after epoch 5:
0.0024486184120178223
In-batch accuracy: 1.0
Test accuracy: 0.81

Loss after epoch 6:
0.029270902276039124
In-batch accuracy: 1.0
Test accuracy: 0.8

Loss after epoch 7:
0.0
In-batch accuracy: 1.0
Test accuracy: 0.88

Loss after epoch 8:
2.9802322387695312e-06
In-batch accuracy: 1.0
Test accuracy: 0.68

Loss after epoch 9:
0.0
In-batch accuracy: 1.0
Test accuracy: 0.82

CPU times: user 1min 47s, sys: 38.8 s, total: 2min 26s
Wall time: 2min 27s


In [10]:
model.eval()

predictions = []
lables = []

for i in np.random.choice(range(25000), TEST_SIZE, replace=False):
    _test = test[i]
#     test_texts.append(_test[0])
#     test_labels.append(_test[1])
    _text = _test[0].unsqueeze(0).permute(0, 2, 1)
    _text = Variable(_text.cuda()) if CUDA else Variable(_text)
    lables.append(_test[1])
    pred = model(_text)
    _, idx = torch.max(pred, 1)
    predictions.append(idx.data[0])

print(accuracy_score(lables, predictions))

model.train()
print('')

0.82

